In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close
from reversible2.bhno import load_file, create_inputs
th.backends.cudnn.benchmark = True

In [ ]:
from hyperoptim.results import load_data_frame, remove_columns_with_same_value, mean_identical_exps, pairwise_compare_frame
import pandas as pd

In [ ]:
def load_df(folder):
    df = load_data_frame(folder)
    df = df.drop(['save_folder', 'seed', 'only_return_exp'], axis=1)
    df = df[df.finished == 1]
    df = df[df.debug == 0]
    if len(df) > 0:
        for key in ['train_acc', 'train_data_acc','test_acc', 'test_data_acc']:
            df.loc[:,key] = np.round(df.loc[:,key] * 100,1)
        df = remove_columns_with_same_value(df) 
        df.runtime = pd.to_timedelta(np.round(df.runtime), unit='s')
    print(len(df))
    return df

In [ ]:
result_cols = ['g_loss', 'ot_in_loss_other_1',
               'runtime', 'test_data_acc', 'test_acc', 'train_data_acc', 'ot_out_loss',
       'ot_in_loss_1', 'train_acc', 'g_grad', 'ot_in_loss_other_0',
       'ot_loss_in_1', 'g_grad_norm', 'ot_in_loss_0', 'ot_loss_in_0',
       'ot_out_loss_other', 'clf_loss_0',  'clf_loss_1','subspace_loss_0',
               'subspace_loss_1', 'test_clf_acc',
              'train_clf_acc']

interesting_result_cols = [
               'runtime', 'test_data_acc', 'test_acc', 'test_clf_acc',  'ot_out_loss',
        'ot_loss_in_0', 'ot_loss_in_1',
       'subspace_loss_0',
               'subspace_loss_1', 
              'train_data_acc', 'train_acc','train_clf_acc', ]

In [ ]:
df = load_df('/data/schirrmr/schirrmr/reversible/experiments/for-poster-2///')
df.loc[:,['subject_id', 'max_epochs',] + 
                                              interesting_result_cols].sort_values(
    by='subject_id')

In [ ]:
sensor_names = [
            'Fz',
            'FC3','FC1','FCz','FC2','FC4',
            'C5','C3','C1','Cz','C2','C4','C6',
            'CP3','CP1','CPz','CP2','CP4',
            'P1','Pz','P2',
            'POz']
orig_train_cnt = load_file('/data/schirrmr/schirrmr/HGD-public/reduced/train/9.mat')
train_cnt = orig_train_cnt.reorder_channels(sensor_names)

train_inputs = create_inputs(train_cnt, final_hz=64, half_before=True,
                            start_ms=500,stop_ms=1500)

In [ ]:
feature_model = th.load('/data/schirrmr/schirrmr/reversible/experiments/for-poster-2/4/feature_model.pkl')
class_dist = th.load('/data/schirrmr/schirrmr/reversible/experiments/for-poster-2/4/class_dist.pkl')

In [ ]:
handles, labels

In [ ]:
ax.get_legend_handles_labels()

In [ ]:
ax.get_legend_handles_labels

In [ ]:
l.get_label()

In [ ]:
from reversible2.plot import plot_head_signals_tight
inverted_per_class = []
for i_class in range(2):
    samples = class_dist.get_mean_std(i_class)[0].unsqueeze(0)
    inverted = feature_model.invert(samples)
    inverted_per_class.append(var_to_np(inverted)[0].squeeze())
fig = plot_head_signals_tight(np.stack(inverted_per_class, axis=-1), sensor_names=sensor_names,
                         figsize=(20,12));
ax = plt.gca()

fig.legend(ax.get_lines()[:2], ["Right Hand", "Resting State"], fontsize=24)

In [ ]:
from reversible2.plot import plot_head_signals_tight
inverted_per_class = []
for i_class in range(2):
    inverted = train_inputs[i_class]
    inverted_per_class.append(var_to_np(inverted)[0].squeeze())
plot_head_signals_tight(np.stack(inverted_per_class, axis=-1), sensor_names=sensor_names,
                         figsize=(20,12));